In [3]:
import pandas as pd
import numpy as np
from collections import Counter

In [4]:
df = pd.read_parquet('LAGT_v3-0.parquet')

In [9]:
len(df)

1710

In [5]:
df.head()

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet,provenience,lemmatacount
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",grecy,None,101.0,200.0,None,[],christian,34
8,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,"[[οὐδενός, εὐνητήρ], [μακροπτολέμοιο, μήτηρ, μ...",grecy,4-3 B.C.,-400.0,-201.0,False,[Bucolici],pagan,61
9,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",grecy,5 B.C.,-500.0,-401.0,False,[Tragici],pagan,10277
10,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,Οἴκοι τὰ Μιλήσια: ἐπὶ τῶν ὅποι μὴ προςήκει τὴν...,2685,1Kgr,"[[Μιλήσιος], [προςήκω, τρυφή, ἐπιδείκνυμι], [Ἀ...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]",pagan,1488
11,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,Κατὰ πετρῶν σπείρεις. Πλίνθον πλύνεις. Δικτύῳ ...,143,1Kgr,"[[πέτρα, σπείρω], [Πλίνθος, πλύνω, Δίκτυον, ἄν...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]",pagan,125


In [6]:
df_filtered = df[df['lemmata_source'] != 'grecy'].copy() ##remove all automated lemmatazationc

In [8]:
len(df_filtered)

948

In [22]:
all_words = df_filtered['lemmatized_sentences'].tolist()

flat_list = []
for sublist in all_words:
    for item in sublist:
        if isinstance(item, np.ndarray):
            item = item.tolist()  
        flat_list.extend(item)  

In [27]:
# Convert Counter to DataFrame
df_word_counts = pd.DataFrame.from_dict(word_counts, orient='index', columns=['Count'])
df_word_counts.index.name = 'Word'

# Reset index to make 'Word' a column
df_word_counts = df_word_counts.reset_index()

df_word_counts = df_word_counts.sort_values(by='Count', ascending=False)

In [28]:
df_word_counts.to_csv('frequency_total_corpus.csv')

In [16]:
def get_author_word_counts(df):
    author_word_counts = {}
    
    # Group by 'author'
    grouped = df.groupby('author')['lemmatized_sentences']
    
    for author, sentences_list in grouped:
        flat_list = []
        
        # Iterate over each list of sentences
        for sublist in sentences_list:
            # Check if sublist is numpy.ndarray
            if isinstance(sublist, np.ndarray):
                sublist = sublist.tolist()
            
            # Extend flat_list with words
            for item in sublist:
                if isinstance(item, np.ndarray):
                    item = item.tolist()
                flat_list.extend(item)
        
        # Count word frequencies
        word_counts = Counter(flat_list)
        author_word_counts[author] = word_counts
        
    all_words = set(word for counts in author_word_counts.values() for word in counts)
    data = {word: {author: counts.get(word, 0) for author, counts in author_word_counts.items()} for word in all_words}
    df_word_counts = pd.DataFrame(data).T.fillna(0)
    
    word_totals = df_word_counts.sum(axis=1)
    
    sorted_words = word_totals.sort_values(ascending=False).index
    
    df_word_counts = df_word_counts.loc[sorted_words]
    
    return df_word_counts
    return author_word_counts

In [17]:
author_matrix = get_author_word_counts(df_filtered)

In [20]:
author_matrix.to_csv('author_matrix.csv')

In [10]:
import pandas as pd
import numpy as np
from collections import Counter

def get_title_word_counts(df):
    title_word_counts = {}
    
    grouped = df.groupby('title')['lemmatized_sentences']

    for title, sentences_list in grouped:
        flat_list = []

        #convert to list
        for sublist in sentences_list:
            if isinstance(sublist, np.ndarray):
                sublist = sublist.tolist()
            
            for item in sublist:
                if isinstance(item, np.ndarray):
                    item = item.tolist()
                flat_list.extend(item)

        
        word_counts = Counter(flat_list)
        title_word_counts[title] = word_counts
        
    all_words = set(word for counts in title_word_counts.values() for word in counts)
    data = {word: {title: counts.get(word, 0) for title, counts in title_word_counts.items()} for word in all_words}
    df_word_counts = pd.DataFrame(data).T.fillna(0)
    
    word_totals = df_word_counts.sum(axis=1)
    
    sorted_words = word_totals.sort_values(ascending=False).index
    
    df_word_counts = df_word_counts.loc[sorted_words]
    
    return df_word_counts

In [11]:
title_matrix = get_title_word_counts(df_filtered)

In [12]:
title_matrix.to_csv('title_matrix.csv')

In [29]:
df_corpus = df[['author', 'title']]

In [30]:
df_corpus.to_csv('corpus_index.csv')